In [1]:
from pathlib import Path
from openeye import oechem, oespruce
import pandas as pd
import tempfile
from typing import List, Tuple
import numpy as np

# The problem

I ran a slightly modified version of JDC's script [found here](https://github.com/RobertArbon/covid-moonshot/blob/dimer/scripts/00-prep-all-receptors.py) to generate outputs so i could make sure my new script reproduced the original results. The results, for x0072 and x0689 can be found in the folder `JDC`. You can @vis...pml the PyMol script to see the dyad in stick form. 

**Problem**: the histidine does not get protonated. I think in the Slack channel I said it was the Cys that doesn't get affected - but that was wrong (or I had another error...) 

This notebook presents the absolute bare minimum to reproduce this problem[1]. The results of running this notebook, `-protein.pdb` and `-protein-thioloate.pdb`, are put in the root of this folder. Also in this folder are PyMol scripts to quickly show the dyad in stick form. 

**Note**  just attempting to protonate the His results in an error which affects the Cys as well. To test this, I stripped the hydrogen from Cys and put it back again. I then attempted to protonate the His.  The result (`-protein-his-plus.pdb`) shows that no protons are put on Cys or His. This is backed up the error log (at the very bottom) which says 

```
Warning: Hydrogens have not been placed, can not add OEInteractionHints
```

I've included a P form model for comparison. The problem persists there as well, with the added complication that Spruce decides to switch the bond order of NE2/ND1. 

All the outputs should have been generated but you should change the 'input_file' and 'is_P' parameters to generate the results again. To reproduce the `protein-his-plus.pdb`, you'll need to see the relevant code below (under **The Problem** sub-section below). 

[1] if you vimdiff the relevant pdbs created by the JDC script and this notebook, there are small differences in atom numbering and position. 

# Parameters

In [2]:
input_file = Path('Mpro-x0072_0A_bound.pdb')
is_P = False

# Setup

In [3]:
SEQRES_MONOMER = """\
SEQRES   1 A  306  SER GLY PHE ARG LYS MET ALA PHE PRO SER GLY LYS VAL
SEQRES   2 A  306  GLU GLY CYS MET VAL GLN VAL THR CYS GLY THR THR THR
SEQRES   3 A  306  LEU ASN GLY LEU TRP LEU ASP ASP VAL VAL TYR CYS PRO
SEQRES   4 A  306  ARG HIS VAL ILE CYS THR SER GLU ASP MET LEU ASN PRO
SEQRES   5 A  306  ASN TYR GLU ASP LEU LEU ILE ARG LYS SER ASN HIS ASN
SEQRES   6 A  306  PHE LEU VAL GLN ALA GLY ASN VAL GLN LEU ARG VAL ILE
SEQRES   7 A  306  GLY HIS SER MET GLN ASN CYS VAL LEU LYS LEU LYS VAL
SEQRES   8 A  306  ASP THR ALA ASN PRO LYS THR PRO LYS TYR LYS PHE VAL
SEQRES   9 A  306  ARG ILE GLN PRO GLY GLN THR PHE SER VAL LEU ALA CYS
SEQRES  10 A  306  TYR ASN GLY SER PRO SER GLY VAL TYR GLN CYS ALA MET
SEQRES  11 A  306  ARG PRO ASN PHE THR ILE LYS GLY SER PHE LEU ASN GLY
SEQRES  12 A  306  SER CYS GLY SER VAL GLY PHE ASN ILE ASP TYR ASP CYS
SEQRES  13 A  306  VAL SER PHE CYS TYR MET HIS HIS MET GLU LEU PRO THR
SEQRES  14 A  306  GLY VAL HIS ALA GLY THR ASP LEU GLU GLY ASN PHE TYR
SEQRES  15 A  306  GLY PRO PHE VAL ASP ARG GLN THR ALA GLN ALA ALA GLY
SEQRES  16 A  306  THR ASP THR THR ILE THR VAL ASN VAL LEU ALA TRP LEU
SEQRES  17 A  306  TYR ALA ALA VAL ILE ASN GLY ASP ARG TRP PHE LEU ASN
SEQRES  18 A  306  ARG PHE THR THR THR LEU ASN ASP PHE ASN LEU VAL ALA
SEQRES  19 A  306  MET LYS TYR ASN TYR GLU PRO LEU THR GLN ASP HIS VAL
SEQRES  20 A  306  ASP ILE LEU GLY PRO LEU SER ALA GLN THR GLY ILE ALA
SEQRES  21 A  306  VAL LEU ASP MET CYS ALA SER LEU LYS GLU LEU LEU GLN
SEQRES  22 A  306  ASN GLY MET ASN GLY ARG THR ILE LEU GLY SER ALA LEU
SEQRES  23 A  306  LEU GLU ASP GLU PHE THR PRO PHE ASP VAL VAL ARG GLN
SEQRES  24 A  306  CYS SER GLY VAL THR PHE GLN
"""
SEQRES_DIMER = """\
SEQRES   1 A  306  SER GLY PHE ARG LYS MET ALA PHE PRO SER GLY LYS VAL
SEQRES   2 A  306  GLU GLY CYS MET VAL GLN VAL THR CYS GLY THR THR THR
SEQRES   3 A  306  LEU ASN GLY LEU TRP LEU ASP ASP VAL VAL TYR CYS PRO
SEQRES   4 A  306  ARG HIS VAL ILE CYS THR SER GLU ASP MET LEU ASN PRO
SEQRES   5 A  306  ASN TYR GLU ASP LEU LEU ILE ARG LYS SER ASN HIS ASN
SEQRES   6 A  306  PHE LEU VAL GLN ALA GLY ASN VAL GLN LEU ARG VAL ILE
SEQRES   7 A  306  GLY HIS SER MET GLN ASN CYS VAL LEU LYS LEU LYS VAL
SEQRES   8 A  306  ASP THR ALA ASN PRO LYS THR PRO LYS TYR LYS PHE VAL
SEQRES   9 A  306  ARG ILE GLN PRO GLY GLN THR PHE SER VAL LEU ALA CYS
SEQRES  10 A  306  TYR ASN GLY SER PRO SER GLY VAL TYR GLN CYS ALA MET
SEQRES  11 A  306  ARG PRO ASN PHE THR ILE LYS GLY SER PHE LEU ASN GLY
SEQRES  12 A  306  SER CYS GLY SER VAL GLY PHE ASN ILE ASP TYR ASP CYS
SEQRES  13 A  306  VAL SER PHE CYS TYR MET HIS HIS MET GLU LEU PRO THR
SEQRES  14 A  306  GLY VAL HIS ALA GLY THR ASP LEU GLU GLY ASN PHE TYR
SEQRES  15 A  306  GLY PRO PHE VAL ASP ARG GLN THR ALA GLN ALA ALA GLY
SEQRES  16 A  306  THR ASP THR THR ILE THR VAL ASN VAL LEU ALA TRP LEU
SEQRES  17 A  306  TYR ALA ALA VAL ILE ASN GLY ASP ARG TRP PHE LEU ASN
SEQRES  18 A  306  ARG PHE THR THR THR LEU ASN ASP PHE ASN LEU VAL ALA
SEQRES  19 A  306  MET LYS TYR ASN TYR GLU PRO LEU THR GLN ASP HIS VAL
SEQRES  20 A  306  ASP ILE LEU GLY PRO LEU SER ALA GLN THR GLY ILE ALA
SEQRES  21 A  306  VAL LEU ASP MET CYS ALA SER LEU LYS GLU LEU LEU GLN
SEQRES  22 A  306  ASN GLY MET ASN GLY ARG THR ILE LEU GLY SER ALA LEU
SEQRES  23 A  306  LEU GLU ASP GLU PHE THR PRO PHE ASP VAL VAL ARG GLN
SEQRES  24 A  306  CYS SER GLY VAL THR PHE GLN
SEQRES   1 B  306  SER GLY PHE ARG LYS MET ALA PHE PRO SER GLY LYS VAL
SEQRES   2 B  306  GLU GLY CYS MET VAL GLN VAL THR CYS GLY THR THR THR
SEQRES   3 B  306  LEU ASN GLY LEU TRP LEU ASP ASP VAL VAL TYR CYS PRO
SEQRES   4 B  306  ARG HIS VAL ILE CYS THR SER GLU ASP MET LEU ASN PRO
SEQRES   5 B  306  ASN TYR GLU ASP LEU LEU ILE ARG LYS SER ASN HIS ASN
SEQRES   6 B  306  PHE LEU VAL GLN ALA GLY ASN VAL GLN LEU ARG VAL ILE
SEQRES   7 B  306  GLY HIS SER MET GLN ASN CYS VAL LEU LYS LEU LYS VAL
SEQRES   8 B  306  ASP THR ALA ASN PRO LYS THR PRO LYS TYR LYS PHE VAL
SEQRES   9 B  306  ARG ILE GLN PRO GLY GLN THR PHE SER VAL LEU ALA CYS
SEQRES  10 B  306  TYR ASN GLY SER PRO SER GLY VAL TYR GLN CYS ALA MET
SEQRES  11 B  306  ARG PRO ASN PHE THR ILE LYS GLY SER PHE LEU ASN GLY
SEQRES  12 B  306  SER CYS GLY SER VAL GLY PHE ASN ILE ASP TYR ASP CYS
SEQRES  13 B  306  VAL SER PHE CYS TYR MET HIS HIS MET GLU LEU PRO THR
SEQRES  14 B  306  GLY VAL HIS ALA GLY THR ASP LEU GLU GLY ASN PHE TYR
SEQRES  15 B  306  GLY PRO PHE VAL ASP ARG GLN THR ALA GLN ALA ALA GLY
SEQRES  16 B  306  THR ASP THR THR ILE THR VAL ASN VAL LEU ALA TRP LEU
SEQRES  17 B  306  TYR ALA ALA VAL ILE ASN GLY ASP ARG TRP PHE LEU ASN
SEQRES  18 B  306  ARG PHE THR THR THR LEU ASN ASP PHE ASN LEU VAL ALA
SEQRES  19 B  306  MET LYS TYR ASN TYR GLU PRO LEU THR GLN ASP HIS VAL
SEQRES  20 B  306  ASP ILE LEU GLY PRO LEU SER ALA GLN THR GLY ILE ALA
SEQRES  21 B  306  VAL LEU ASP MET CYS ALA SER LEU LYS GLU LEU LEU GLN
SEQRES  22 B  306  ASN GLY MET ASN GLY ARG THR ILE LEU GLY SER ALA LEU
SEQRES  23 B  306  LEU GLU ASP GLU PHE THR PRO PHE ASP VAL VAL ARG GLN
SEQRES  24 B  306  CYS SER GLY VAL THR PHE GLN
"""

MINIMUM_FRAGMENT_SIZE = 7

In [4]:
errfs = oechem.oeosstream() # create a stream that writes internally to a stream
oechem.OEThrow.SetOutputStream(errfs)
oechem.OEThrow.Clear()
oechem.OEThrow.SetLevel(oechem.OEErrorLevel_Verbose)

In [5]:
def read_pdb_file(pdb_file):
    ifs = oechem.oemolistream()
    ifs.SetFlavor(oechem.OEFormat_PDB, oechem.OEIFlavor_PDB_Default | oechem.OEIFlavor_PDB_DATA | oechem.OEIFlavor_PDB_ALTLOC)  # noqa

    if not ifs.open(pdb_file):
        oechem.OEThrow.Fatal("Unable to open %s for reading." % pdb_file)

    mol = oechem.OEGraphMol()
    if not oechem.OEReadMolecule(ifs, mol):
        oechem.OEThrow.Fatal("Unable to read molecule from %s." % pdb_file)
    ifs.close()

    return mol

def remove_from_lines(lines: List[str], string: str) -> List[str]:
    return [line for line in lines if string not in line]

def is_in_lines(lines: List[str], string: str) -> bool:
    return np.any([string in line for line in lines])


def add_prefix(lines: List[str], prefix: str) -> List[str]:
    return [line + '\n' for line in prefix.split('\n')] + lines


def lines_to_mol_graph(lines: List[str]) -> oechem.OEGraphMol:

    with tempfile.NamedTemporaryFile(delete=False, mode='wt', suffix='.pdb') as pdbfile:
        pdbfile.write(''.join(lines))
        pdbfile.close()
        mol = read_pdb_file(pdbfile.name)

    return mol

def change_protonation(molecule: oechem.OEGraphMol, options: oechem.OEPlaceHydrogensOptions,
                       match_strings: List[str], atom_name: int, formal_charge: int, implicit_h_count: int) -> \
        Tuple[oechem.OEGraphMol, oechem.OEPlaceHydrogensOptions]:
    pred = oechem.OEAtomMatchResidue(match_strings)
    options.SetBypassPredicate(pred)
    for atom in molecule.GetAtoms(pred):
        if oechem.OEGetPDBAtomIndex(atom) == atom_name:
            print(f'\t ...changing protonation on {atom}')
            oechem.OESuppressHydrogens(atom)
            atom.SetImplicitHCount(implicit_h_count)
            atom.SetFormalCharge(formal_charge)
    return molecule, options

# Calculation

In [6]:
if is_P: 
    seqres = SEQRES_DIMER
else:
    seqres = SEQRES_MONOMER

Remove extraneous bits from PDB

In [7]:
lines = input_file.open().readlines()
lines = remove_from_lines(lines, 'UNK')
lines = remove_from_lines(lines, 'HOH')
lines = remove_from_lines(lines, 'LINK')
if not is_in_lines(lines, 'SEQRES'):
    lines = add_prefix(lines, seqres)
mol = lines_to_mol_graph(lines)

Set options

In [8]:
opts = oespruce.OEMakeDesignUnitOptions()

opts.GetSplitOptions().SetMinLigAtoms(MINIMUM_FRAGMENT_SIZE) # minimum fragment size (in heavy atoms)
opts.GetPrepOptions().SetStrictProtonationMode(True)

opts.GetPrepOptions().GetBuildOptions().SetCapNTermini(False)
opts.GetPrepOptions().GetBuildOptions().SetCapCTermini(False)
opts.GetPrepOptions().GetBuildOptions().SetBuildLoops(True)
opts.GetPrepOptions().GetBuildOptions().SetBuildSidechains(True)

opts.GetPrepOptions().GetBuildOptions().GetCapBuilderOptions().SetAllowTruncate(False)

# premptive measure by JDC - not sure whether this is actually needed.
# opts = prevent_flip(opts, match_strings=["GLN:189:.*:.*:.*"])

metadata = oespruce.OEStructureMetadata()
protonate_opts = opts.GetPrepOptions().GetProtonateOptions()
place_h_opts = protonate_opts.GetPlaceHydrogensOptions()


Strip hydrogens

In [9]:
for atom in mol.GetAtoms():
    if atom.GetAtomicNum() > 1:
        oechem.OESuppressHydrogens(atom)

Make design units. Note, Spruce decides that bond orders of NE2/ND1 of histodine should be switched in the case of the P series.  

In [10]:
dus = list(oespruce.OEMakeDesignUnits(mol, metadata, opts))

protein = oechem.OEGraphMol()
dus[0].GetProtein(protein)

True

Write protein before changing protonation.  

In [11]:
with oechem.oemolostream(str(input_file).replace('.pdb', '-protein.pdb')) as ofs:
    oechem.OEWriteMolecule(ofs, oechem.OEGraphMol(protein))

## The problem

Now make the charge separated dyad. As written there is a problem - the hydrogens on the his do not get added. They also the affect the protonation of the Cys. To see this, force the Sulfur atom have a charge 0 and 1 hydrogen atom (see the comments in the next two cells)


The protonation step can be wrapped in the 'change_protonation' function (see second comment below) should you want to experiment. 

In [12]:
pred = oechem.OEAtomMatchResidue(["CYS:145:.*:.*:.*"])
place_h_opts.SetBypassPredicate(pred)
for atom in protein.GetAtoms(pred):
    if oechem.OEGetPDBAtomIndex(atom) == oechem.OEPDBAtomName_SG:
        print(f'\t ...changing protonation on {atom}')
        oechem.OESuppressHydrogens(atom)
        atom.SetFormalCharge(-1)
        atom.SetImplicitHCount(0)
#         Uncomment to force thiol
#         atom.SetFormalCharge(0)
#         atom.SetImplicitHCount(1)


# protein, place_h_opts = change_protonation(molecule=protein, options=place_h_opts, match_strings=["HIS:41:.*:.*:.*"], 
#                                            atom_name=oechem.OEPDBAtomName_ND1, formal_charge=+1, implicit_h_count=1)
pred = oechem.OEAtomMatchResidue(["HIS:41:.*:.*:.*"])
place_h_opts.SetBypassPredicate(pred)
for atom in protein.GetAtoms(pred):
    if oechem.OEGetPDBAtomIndex(atom) == oechem.OEPDBAtomName_ND1:
        print(f'\t ...changing protonation on {atom}')
        oechem.OESuppressHydrogens(atom) # strip hydrogens from residue
        atom.SetFormalCharge(+1)
        atom.SetImplicitHCount(1)



oechem.OEUpdateDesignUnit(dus[0], protein, oechem.OEDesignUnitComponents_Protein)
oespruce.OEProtonateDesignUnit(dus[0], protonate_opts)

	 ...changing protonation on 1119 S
	 ...changing protonation on 308 N


True

In [13]:
with oechem.oemolostream(str(input_file).replace('.pdb', '-protein-thiolate.pdb')) as ofs:
    oechem.OEWriteMolecule(ofs, oechem.OEGraphMol(protein))

# Uncomment to write the thiol with protonated histidine
# with oechem.oemolostream(str(input_file).replace('.pdb', '-protein-his-plus.pdb')) as ofs:
#     oechem.OEWriteMolecule(ofs, oechem.OEGraphMol(protein))

Generate pymol script. This adds the protein with the neutral and charged dyad states.  simply run @vis_x/P....pml from PyMol to visualise problem. 

In [14]:
lines = ['reinitialize', f"load {str(input_file).replace('.pdb', '-protein.pdb')}, netural",
        f"load {str(input_file).replace('.pdb', '-protein-thiolate.pdb')}, charged", 
        'hide cartoon', 'show sticks, resi 41 or resi 145']
with open(f'vis_{str(input_file.stem)}.pml', 'wt') as f:
    f.writelines('\n'.join(lines))

Inspect errors: 

In [15]:
print(errfs)

DPI: 0.11, RFree: 0.23, Resolution: 1.65
Processing BU # 1 with title: , chains A, alt: A
Processing BU # 2 with title: , chains A, alt: B
DU: (A)altA > LIG(A-1101), Iridium Category: NA, LaD: 0.00, ASaD: 0.00, DPI: 0.11, POL: true, POAS: true, AltConfs: false, PackRes: false, Excp: false, IrrRFree: false, PossCov: false
DU: (A)altB > LIG(A-1101), Iridium Category: NA, LaD: 0.00, ASaD: 0.00, DPI: 0.11, POL: true, POAS: true, AltConfs: false, PackRes: false, Excp: false, IrrRFree: false, PossCov: false
Skipping redundant DU with alts outside the site of interest, renaming existing to collapse alts
Discarding redundant alt DU with title (A)altB > LIG(A-1101)
DU: (A) > LIG(A-1101), Iridium Category: NA, LaD: 0.00, ASaD: 0.00, DPI: 0.11, POL: true, POAS: true, AltConfs: false, PackRes: false, Excp: false, IrrRFree: false, PossCov: false

